In [1]:
import torch

c:\Users\Berry\.conda\envs\dmn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import torch

# Base class definition for demonstration
class BaseElement:
    pass

# Provided code for TriangularElement
class TriangularElement(BaseElement):
    element_dof = 6
    node_per_element = 3

    @staticmethod
    def shape_functions(natural_coords: torch.Tensor, device='cuda') -> torch.Tensor:
        xi, eta = natural_coords
        N = torch.tensor([xi, eta, 1 - xi - eta]).to(device)
        return N

    @staticmethod
    def shape_function_derivatives(device='cuda') -> torch.Tensor:
        dN_dxi = torch.tensor([1, 0, -1], device=device)
        dN_deta = torch.tensor([0, 1, -1], device=device)
        dN = torch.stack((dN_dxi, dN_deta), dim=0)
        return dN

    @staticmethod
    def compute_B_matrix_legacy(node_coords: torch.Tensor, device='cuda') -> torch.Tensor:
        if len(node_coords.shape) != 2 or node_coords.shape[0] != 3 or node_coords.shape[1] != 2:
            raise ValueError("node_coords shape should be (3, 2) for a single triangle")
        x1, y1 = node_coords[0]
        x2, y2 = node_coords[1]
        x3, y3 = node_coords[2]
        A = 0.5 * (x1*(y2 - y3) + x2*(y3 - y1) + x3*(y1 - y2))
        if abs(A) < 1e-10:
            raise ValueError("The provided node coordinates result in a degenerate triangle with almost zero area.")
        b1 = (y2 - y3) / (2*A)
        b2 = (y3 - y1) / (2*A)
        b3 = (y1 - y2) / (2*A)
        c1 = (x3 - x2) / (2*A)
        c2 = (x1 - x3) / (2*A)
        c3 = (x2 - x1) / (2*A)
        B = torch.tensor([
            [b1, 0, b2, 0, b3, 0],
            [0, c1, 0, c2, 0, c3],
            [c1, b1, c2, b2, c3, b3]
        ], dtype=torch.float).to(device)
        return B

# Modified code for T3Element
class T3Element:
    element_dof = 6
    node_per_element = 3

    @staticmethod
    def shape_functions(natural_coords: torch.Tensor, device='cuda') -> torch.Tensor:
        xi, eta = natural_coords
        N = torch.tensor([1 - xi - eta, xi, eta]).to(device)
        return N

    @staticmethod
    def shape_function_derivatives(device='cuda') -> torch.Tensor:
        dN_dxi = torch.tensor([-1, 1, 0], device=device, dtype=torch.float)
        dN_deta = torch.tensor([-1, 0, 1], device=device, dtype=torch.float)
        dN = torch.stack((dN_dxi, dN_deta), dim=0)
        return dN

    @staticmethod
    def compute_B_matrix(node_coords: torch.Tensor, device='cuda') -> torch.Tensor:
        dN_dxi = T3Element.shape_function_derivatives(device)
        J = torch.mm(dN_dxi, node_coords)
        detJ = torch.det(J)
        if detJ == 0:
            raise ValueError("The Jacobian is singular")
        inv_J = torch.inverse(J)
        dN_dxy = torch.mm(inv_J, dN_dxi)
        B = torch.zeros(3, 6, device=device)
        B[0, 0::2] = dN_dxy[0, :]
        B[1, 1::2] = dN_dxy[1, :]
        B[2, 0::2] = dN_dxy[1, :]
        B[2, 1::2] = dN_dxy[0, :]
        return B

# Generate test data
node_coords = torch.tensor([[1.0, 1.0], [3.0, 1.0], [2.0, 4.0]], device='cuda')
natural_coords = torch.tensor([0.2, 0.3], device='cuda')

# Compute B-matrix using both classes
B_T3 = T3Element.compute_B_matrix(node_coords)
B_Triangular = TriangularElement.compute_B_matrix_legacy(node_coords)

# Display the results
B_T3, B_Triangular, torch.allclose(B_T3, B_Triangular)


(tensor([[-0.5000,  0.0000,  0.5000,  0.0000,  0.0000,  0.0000],
         [ 0.0000, -0.1667,  0.0000, -0.1667,  0.0000,  0.3333],
         [-0.1667, -0.5000, -0.1667,  0.5000,  0.3333,  0.0000]],
        device='cuda:0'),
 tensor([[-0.5000,  0.0000,  0.5000,  0.0000,  0.0000,  0.0000],
         [ 0.0000, -0.1667,  0.0000, -0.1667,  0.0000,  0.3333],
         [-0.1667, -0.5000, -0.1667,  0.5000,  0.3333,  0.0000]],
        device='cuda:0'),
 True)

In [10]:
import numpy as np
from scipy.special import roots_legendre, eval_legendre
roots_legendre(2)

(array([-0.57735027,  0.57735027]), array([1., 1.]))